In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib


from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

In [45]:
def tts_strat_cont(X, y, train_size, val_size, test_size, random_state):
    
    # Stratify based on y using pd.qcut
    y_bin = pd.qcut(y, q=5)

    # First split, stratify on y_bin
    X_train, X_other, y_train, y_other = train_test_split(
        X, y, train_size=train_size, stratify=y_bin, random_state=random_state)

    # Scale the validation size to reflect the remaining data
    val_size_scaled = val_size / (val_size + test_size)

    # Create stratified bins for the second split
    y_bin_other = pd.qcut(y_other, q=5)
    
    # Second split, stratify on y_bin_other
    X_val, X_test, y_val, y_test = train_test_split(
        X_other, y_other, train_size=val_size_scaled, stratify=y_bin_other, random_state=random_state)

    return X_train, y_train, X_val, y_val, X_test, y_test

In [57]:
best_models = []
test_scores = []

# function for the ML pipeline
def MLpipe_Strat_RMSE(X, y, train_size, val_size, test_size, preprocessor, ML_algo, param_grid, random_states):
    
        for i in range(random_states):
            
            try:
                # Instantiate the model with random_state if applicable
                ML_algo_instance = ML_algo(random_state=i)
            except TypeError:
                # If random_state is not an argument, use the model as is
                ML_algo_instance = ML_algo
            
            try:
                # Set max_iter if it's an argument
                ML_algo_instance.set_params(max_iter=1000000)
            except (TypeError, ValueError):
                # If max_iter is not an argument, just leave it as is
                pass
    
            # call my split function
            X_train, y_train, X_val, y_val, X_test, y_test = tts_strat_cont(X, y, train_size, val_size, test_size, random_state=i)
    
            # preprocess the sets and fit model with inputs
            pipe = make_pipeline(preprocessor, ML_algo) 
            # Exhaustive search over specified parameter values
            grid = GridSearchCV(pipe, param_grid=param_grid, scoring='neg_root_mean_squared_error', 
                                return_train_score=True, n_jobs=-1, cv=[(np.arange(len(X_train)), np.arange(len(X_val)))],
                                verbose=True)
            grid.fit(X_train, y_train)

            # access results and save them into a data frame
            results = pd.DataFrame(grid.cv_results_)
            #print(results)
            # access best models and save them
            best_models.append(grid.best_estimator_)

            #print('best model parameters:',grid.best_params_)
            #print('validation score:',grid.best_score_) # this is the mean validation score over all iterations
            y_test_pred = grid.predict(X_test)
            test_scores.append(np.sqrt(mean_squared_error(y_test,y_test_pred)))
             #print("")

        return test_scores, best_models


In [47]:
df = pd.read_csv("processed_data_1/merge.csv")
df = df.drop(df.columns[0], axis=1)

# add a random column to test the coefficients
df["rand"] = np.random.randint(1, 6, df.shape[0])


In [48]:

X = df[['no_wrk_aux','no2_wrk_aux','o3_wrk_aux','temp','rh','t_since_depl','rand']]
y = df['no2_ref']

train_size = 0.6
val_size = 0.2
test_size = 0.2

# collect which encoder to use on each feature
num_cols = X.columns

# collect all the encoders into one preprocessor
num_pipeline= Pipeline([('imputer', SimpleImputer(strategy='median')),
                        ('poly_features', PolynomialFeatures(degree=6, include_bias=False, interaction_only=True)),
                        ('scaler', StandardScaler())])

preprocessor = ColumnTransformer([("num_pipeline", num_pipeline, num_cols)])


In [49]:
# RIDGE

# hyperparameters to tune
param_grid = {
    'ridge__alpha': [0, 0.1, 1, 10, 100]} 

# the model
ML_algo = Ridge() 

# Execute the pipeline
test_scores, best_models = MLpipe_Strat_RMSE(
    X, y, train_size=0.6, val_size=0.2, test_size=0.2, 
    preprocessor=preprocessor, ML_algo=ML_algo, param_grid=param_grid, random_states=5
)

print("Mean of test scores:", np.mean(test_scores))
print("Standard deviation of test scores:", np.std(test_scores))

Fitting 1 folds for each of 5 candidates, totalling 5 fits
Fitting 1 folds for each of 5 candidates, totalling 5 fits
Fitting 1 folds for each of 5 candidates, totalling 5 fits
Fitting 1 folds for each of 5 candidates, totalling 5 fits
Fitting 1 folds for each of 5 candidates, totalling 5 fits
Mean of test scores: 2.3436088793107968
Standard deviation of test scores: 0.03226372346460557


In [56]:
# LASSO 

# hyperparameters to tune
param_grid = {
    'lasso__alpha': [1e-3, 0.1, 1, 10]} 

# the model
ML_algo = Lasso() 

# Execute the pipeline
test_scores, best_models = MLpipe_Strat_RMSE(
    X, y, train_size=0.6, val_size=0.2, test_size=0.2, 
    preprocessor=preprocessor, ML_algo=ML_algo, param_grid=param_grid, random_states=5
)

print("Mean of test scores:", np.mean(test_scores))
print("Standard deviation of test scores:", np.std(test_scores))

Fitting 1 folds for each of 4 candidates, totalling 4 fits
Fitting 1 folds for each of 4 candidates, totalling 4 fits
Fitting 1 folds for each of 4 candidates, totalling 4 fits
Fitting 1 folds for each of 4 candidates, totalling 4 fits
Fitting 1 folds for each of 4 candidates, totalling 4 fits
Mean of test scores: 3.330250142884149
Standard deviation of test scores: 1.6754661985617625


In [59]:
# Elastic Net 

# hyperparameters to tune
param_grid = {
    'elasticnet__alpha': [1e-3, 0.1, 1, 10], 
    'elasticnet__l1_ratio': [0.01, 0.1, 0.5, 0.99]}

# the model
ML_algo = ElasticNet() 

# Execute the pipeline
test_scores, best_models = MLpipe_Strat_RMSE(
    X, y, train_size=0.6, val_size=0.2, test_size=0.2, 
    preprocessor=preprocessor, ML_algo=ML_algo, param_grid=param_grid, random_states=5
)

print("Mean of test scores:", np.mean(test_scores))
print("Standard deviation of test scores:", np.std(test_scores))

Fitting 1 folds for each of 16 candidates, totalling 16 fits


/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.899e+01, tolerance: 1.869e+01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.373e+02, tolerance: 1.869e+01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

Fitting 1 folds for each of 16 candidates, totalling 16 fits


/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.629e+02, tolerance: 1.833e+01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.411e+03, tolerance: 1.833e+01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

Fitting 1 folds for each of 16 candidates, totalling 16 fits


...

Fitting 1 folds for each of 16 candidates, totalling 16 fits


/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.161e+01, tolerance: 1.870e+01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.686e+02, tolerance: 1.870e+01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

Fitting 1 folds for each of 16 candidates, totalling 16 fits


/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.293e+02, tolerance: 1.819e+01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.925e+03, tolerance: 1.819e+01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/data1030/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

Mean of test scores: 2.4286267305081197
Standard deviation of test scores: 0.04934180076781664


In [54]:
# RANDOM FOREST

from sklearn.ensemble import RandomForestRegressor

# parameter grid
param_grid = {
    'randomforestregressor__n_estimators': [1, 3, 10, 30],
    'randomforestregressor__max_depth': [1, 2, 3, 10, 30]} 

# model
ML_algo = RandomForestRegressor()

# Execute the pipeline
test_scores, best_models = MLpipe_Strat_RMSE(
    X, y, train_size=0.6, val_size=0.2, test_size=0.2, 
    preprocessor=preprocessor, ML_algo=ML_algo, param_grid=param_grid, random_states=5
)

print("Mean of test scores:", np.mean(test_scores))
print("Standard deviation of test scores:", np.std(test_scores))

Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Mean of test scores: 3.7004544459545285
Standard deviation of test scores: 1.8571062177088626


In [53]:
# SUPPORT VECTOR MACHINE

# ADD KERNEL AS A HYPERPARAMETER

from sklearn.svm import SVR

# parameter grid
param_grid = {
    'svr__gamma': [1e-3, 1e-1, 1e1, 1e3, 1e5],
    'svr__C': [1e-2, 1e-1, 1e0, 1e1, 1e2]} 

# model
ML_algo = SVR() # no random state

# Execute the pipeline
test_scores, best_models = MLpipe_Strat_RMSE(
    X, y, train_size=0.6, val_size=0.2, test_size=0.2, 
    preprocessor=preprocessor, ML_algo=ML_algo, param_grid=param_grid, random_states=5
)
print("Mean of test scores:", np.mean(test_scores))
print("Standard deviation of test scores:", np.std(test_scores))

Fitting 1 folds for each of 25 candidates, totalling 25 fits
Fitting 1 folds for each of 25 candidates, totalling 25 fits
Fitting 1 folds for each of 25 candidates, totalling 25 fits
Fitting 1 folds for each of 25 candidates, totalling 25 fits
Fitting 1 folds for each of 25 candidates, totalling 25 fits
Mean of test scores: 4.719684132779699
Standard deviation of test scores: 1.432976861294911


In [52]:
# NEAREST NEIGHBOR
from sklearn.neighbors import KNeighborsRegressor

# parameter grid
param_grid = {
    'kneighborsregressor__n_neighbors': [1, 3, 10, 30],
    'kneighborsregressor__weights': ["distance", "uniform"]} 

# model
ML_algo = KNeighborsRegressor() # no random state

# Execute the pipeline
test_scores, best_models = MLpipe_Strat_RMSE(
    X, y, train_size=0.6, val_size=0.2, test_size=0.2, 
    preprocessor=preprocessor, ML_algo=ML_algo, param_grid=param_grid, random_states=5
)
print("Mean of test scores:", np.mean(test_scores))
print("Standard deviation of test scores:", np.std(test_scores))

Fitting 1 folds for each of 8 candidates, totalling 8 fits
Fitting 1 folds for each of 8 candidates, totalling 8 fits
Fitting 1 folds for each of 8 candidates, totalling 8 fits
Fitting 1 folds for each of 8 candidates, totalling 8 fits
Fitting 1 folds for each of 8 candidates, totalling 8 fits
Mean of test scores: 3.2886321978432127
Standard deviation of test scores: 0.09499530125248835
